In [1]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver_properties import auto_test_driver_properties
from pyvisa import ResourceManager, VisaIOError
import pytest
from time import sleep

In [2]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

GPIB0::8::INSTR Stanford_Research_Systems,SR830,s/n86813,ver1.07 



In [17]:
srs830 = ps.Stanford830(res)
auto_test_driver_properties(srs830, verbose=False)

input_configuration
time_constant
amplitude
sensitivity

Beginning tests for:  Stanford830  version  1.0.3
phase
reference_source
frequency
reference_slope
harmonic
input_ground
input_coupling
input_line_filter
reserve_mode
filter_slope
synchronous_filter
display1_output_source
display2_output_source
auxiliary_voltage1
auxiliary_voltage2
auxiliary_voltage3
auxiliary_voltage4
sample_rate
end_buffer_mode
trigger_mode

7 range properties found and tested out of 22 total settings found.
0 values properties found and tested out of 22 total settings found.
13 indexed values properties found and tested out of 22 total settings found.
0 dict values properties found and tested out of 22 total settings found.
4 blacklisted settings not testing (likely due to interdependencies not suitable for automated testing)
20 properties tested out of 22 total settings.
 Property implementation tests passed, instrument: Stanford830. 
Testing driver doc string.
 Docstring tests passed. 
The new test log for t

In [18]:
# Test get and set display

srs830.set_display(1, 'r', 'aux1') # number, source, ratio
source, ratio = srs830.get_display(1)
assert source == 'r'
assert ratio == 'aux1'


srs830.set_display(1, 'x', 'none')
source, ratio = srs830.get_display(1)
assert source == 'x'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(1, 'y', 'none')
print('Sucessfully failed setting channel 1 to y')
    
    
srs830.set_display(2, 'theta', 'aux3')
source, ratio = srs830.get_display(2)
assert source == 'theta'
assert ratio == 'aux3'


srs830.set_display(2, 'y', 'none')
source, ratio = srs830.get_display(2)
assert source == 'y'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(2, 'x', 'none')
print('Sucessfully failed setting channel 2 to x')

print("Successfully tested get and set display")


Sucessfully failed setting channel 1 to y
Sucessfully failed setting channel 2 to x
Successfully tested get and set display


In [19]:
# Test get and set offset and expand

srs830.set_channel_offset_expand(1, 100, 10)
print(srs830.get_channel_offset_expand(1))

srs830.set_channel_offset_expand(1, 0, 1)
print(srs830.get_channel_offset_expand(1))


['100', '1']
['0', '0']


In [20]:
# Test auto offset
# v0 should be some value, v_offset should be close to 0
srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('x')
srs830.auto_offset('x')
v_offset = srs830.read('x')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(2, 0, 1)
v0 = srs830.read('y')
srs830.auto_offset('y')
v_offset = srs830.read('y')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001

srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('r')
srs830.auto_offset('r')
v_offset = srs830.read('r')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(1, 0, 1)
srs830.set_display(1, 'x', 'none')

srs830.set_channel_offset_expand(2, 0, 1)
srs830.set_display(2, 'y', 'none')


-0.000105858 -7.15259e-07
0.000421764 4.76841e-07
3.88625e-05 0.0


In [21]:
# Test read aux inputs
# shoudl show some random small value
for i in range(1, 5):
    aux_input = srs830.read_aux_input(i)
    print(aux_input)
    assert abs(aux_input - 0) < 0.1

0.002
0.001
0.00166667
0.000666667


In [22]:
# Read x, y, r, theta

print(srs830.read('x'))
print(srs830.read('y'))
print(srs830.read('r'))
print(srs830.read('theta'))

-0.000106097
0.000421764
-4.76841e-07
5962.089260533679


In [23]:
# Read display

for i in range(1, 3):
    print(srs830.read_display(i))

-0.000105858
0.000421526


In [24]:
# Test snap

for source in ['y', 'r', 'theta', 'aux1', 'aux2', 'aux3',
                  'aux4', 'frequency', 'display1', 'display2']:
    print(srs830.snap('x', source))

srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1')


with pytest.raises(AssertionError):
    srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1', 'display2')
    # snap only accepts 6 entries

[-0.000105858, 0.000421287]
[-0.000105858, -2.3842e-07]
[-0.000105858, 5963.46436040349]
[-0.000105858, 0.002]
[-0.000105858, 0.004]
[-0.000105858, 0.00433333]
[-0.000105858, 0.00433333]
[-0.000106097, 1000.0]
[-0.000106097, -0.000106097]
[-0.000105858, 0.000421287]


In [25]:
# test get ID

id = srs830.get_identificaiton_string()
assert 'Stanford_Research_Systems,SR830' in id
print(id)

Stanford_Research_Systems,SR830,s/n86813,ver1.07 


In [26]:
# test buffer_points

srs830.buffer_points
srs830.wait_for_trigger()
srs830.set_buffer_mode(8)
srs830.wait_for_trigger()
# srs830.start()
srs830.trigger()
sleep(20)
srs830.buffer_points
srs830.pause()
print(srs830.buffer_points)

159
